In [ ]:
!nvidia-smi
!pip -q install timm transformers accelerate safetensors deepspeed bitsandbytes peft

Wed Nov  5 00:50:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login --token hf_xxxxxxxxxxxxxxxxxxx

In [ ]:
"""

# 저장 폴더 생성
!mkdir -p /content/drive/MyDrive/vla_datasets


# 2) hub 버전은 transformers(4.57.1)와 호환되는 <1.0 유지
!pip -q install "huggingface_hub==0.36.0"

# 3) Python API로 필요한 폴더만 다운로드
from huggingface_hub import snapshot_download
import os

target_dir = "/content/drive/MyDrive/vla_datasets"  # 원하는 저장 위치
os.makedirs(target_dir, exist_ok=True)

snapshot_download(
    repo_id="ShuaiYang03/VLA_Instruction_Tuning",
    repo_type="dataset",
    local_dir=target_dir,
    local_dir_use_symlinks=False,      # Drive 호환
)

print("다운로드 완료:", target_dir)

"""

'\n\n# 저장 폴더 생성\n!mkdir -p /content/drive/MyDrive/vla_datasets\n\n\n# 2) hub 버전은 transformers(4.57.1)와 호환되는 <1.0 유지\n!pip -q install "huggingface_hub==0.36.0"\n\n# 3) Python API로 필요한 폴더만 다운로드\nfrom huggingface_hub import snapshot_download\nimport os\n\ntarget_dir = "/content/drive/MyDrive/vla_datasets"  # 원하는 저장 위치\nos.makedirs(target_dir, exist_ok=True)\n\nsnapshot_download(\n    repo_id="ShuaiYang03/VLA_Instruction_Tuning",\n    repo_type="dataset",\n    local_dir=target_dir,\n    local_dir_use_symlinks=False,      # Drive 호환\n)\n\nprint("다운로드 완료:", target_dir)\n\n'

In [ ]:
import os, pathlib

DRIVE_ROOT = '/content/drive'
MYDRIVE = f'{DRIVE_ROOT}/MyDrive'  # 한국어 UI에서도 내부 폴더명은 MyDrive
print('exists:', os.path.exists(MYDRIVE))

# 작업 폴더(예: vla_project)를 드라이브에 만들고 이동
PROJ_DIR = f'{MYDRIVE}/vla_project'
os.makedirs(PROJ_DIR, exist_ok=True)
os.chdir(PROJ_DIR)
pathlib.Path('.').resolve()

exists: True


PosixPath('/content/drive/MyDrive/vla_project')

In [ ]:
# 프로젝트 루트 지정
BASE = "/content/drive/MyDrive/vla_project"

# 필요한 디렉터리 전부 생성
import os
for d in [
    BASE,
    f"{BASE}/data",
    f"{BASE}/models",
    f"{BASE}/utils",
    f"{BASE}/checkpoints",
]:
    os.makedirs(d, exist_ok=True)

# __init__.py 더미 생성(패키지 인식용)
for f in [
    f"{BASE}/__init__.py",
    f"{BASE}/data/__init__.py",
    f"{BASE}/models/__init__.py",
    f"{BASE}/utils/__init__.py",
]:
    if not os.path.exists(f):
        with open(f, "w", encoding="utf-8") as fp:
            fp.write("# package\n")

In [ ]:
%%writefile /content/drive/MyDrive/vla_project/config.py
from dataclasses import dataclass

@dataclass
class TrainConfig:
    # -----------------------------
    # Data (VLA RLDS / TFDS)
    # -----------------------------
    # TFDS builder directory (Bridge / Fractal)
    builder_dir: str = "/content/drive/MyDrive/vla_datasets/bridge_dataset/1.0.0"
    # reasoning json (Bridge instruction)
    reasoning_json: str = "/content/drive/MyDrive/vla_datasets/annotation/bridge_instruction.json"
    # train / val split ratios
    train_split: str = "train[:80%]"
    val_split: str   = "train[80%:]"
    # image resize
    img_size: int = 224

    # dataloader options
    batch_size: int = 32
    val_batch_size: int = 32
    num_workers: int = 0
    pin_memory: bool = True

    viz_episode_id: str | None = None              # 시각화할 episode_id (None이면 자동탐색)
    viz_max_frames_per_episode: int = 128          # 에피소드 최대 프레임

    # -----------------------------
    # Logging / Checkpoints
    # -----------------------------
    ckpt_dir: str = "checkpoints"
    log_interval: int = 50
    eval_interval: int = 500
    save_interval: int = 1000
    viz_interval: int = 5

    # -----------------------------
    # Model / Backbones
    # -----------------------------
    img_channels: int = 3
    img_size: int = 518
    vision_backbone: str = "vit_base_patch14_dinov2"  # timm backbone
    text_model_name: str = "meta-llama/Meta-Llama-3-8B-Instruct"

    #rl
    rl_state_dim: int = 7
    rl_action_dim: int = 7
    rl_hidden_dim: int = 1024

    # embedding / dims
    vision_embed_dim: int = 768
    text_embed_dim: int = 4096
    state_dim: int = 7
    action_dim: int = 7
    hidden_dim: int = 1024

    # tokenizer settings
    text_max_len: int = 256
    pad_token_str: str = "<|pad|>"

    # -----------------------------
    # LoRA (for LLM fine-tuning)
    # -----------------------------
    lora_r: int = 8
    lora_alpha: int = 32
    lora_dropout: float = 0.1

    # -----------------------------
    # Training
    # -----------------------------
    epochs: int = 10
    lr: float = 3e-4
    weight_decay: float = 1e-4
    amp: bool = True
    grad_clip: float = 1.0
    seed: int = 42

    # -----------------------------
    # DeepSpeed / Distributed
    # -----------------------------
    use_deepspeed: bool = False
    deepspeed_config: str = "ds_config.json"
    grad_accumulation_steps: int = 1

Overwriting /content/drive/MyDrive/vla_project/config.py


In [ ]:
# -----------------------------
# file: utils.py
# -----------------------------

%%writefile /content/drive/MyDrive/vla_project/util.py


import os, random
import torch
import numpy as np

def set_seed(seed: int = 42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

def save_checkpoint(path, state_dict):
  os.makedirs(os.path.dirname(path), exist_ok=True)
  torch.save(state_dict, path)

Overwriting /content/drive/MyDrive/vla_project/util.py


In [ ]:
%%writefile /content/drive/MyDrive/vla_project/utils/vis.py

# utils/vis.py
import re
import torch
@torch.no_grad()
def viz_episode(model_like, loader, device, target_episode_id, out_mp4, max_frames=256):
    """
    loader에서 target_episode_id인 샘플만 모아 step 오름차순으로 정렬 후,
    추론하여 mp4로 저장.
    """
    model_like.eval()
    from utils.vis import make_preview_tile, VideoLogger
    vlog = VideoLogger(out_mp4=out_mp4, fps=4)

    def _to_py(x):
        if isinstance(x, torch.Tensor):
            x = x.detach().cpu()
            return x.item() if x.numel() == 1 else x.tolist()
        return x

    def _get_step(i, aux):
        # aux['step'|'t'|'frame_idx'] 우선, 없으면 file_path에서 숫자 파싱
        for k in ("step","t","frame_idx","index"):
            if k in aux and aux[k] is not None:
                v = aux[k][i]
                if isinstance(v, torch.Tensor):
                    v = v.detach().cpu().item() if v.numel()==1 else int(v.detach().cpu().tolist())
                try: return int(v)
                except: pass
        fp = aux.get("file_path", None)
        if fp is not None:
            s = fp[i] if isinstance(fp, (list,tuple)) else fp
            s = str(s)
            nums = re.findall(r"\d+", s)
            if nums:
                try: return int(nums[-1])
                except: return None
        return None

    # 1) target episode만 수집
    bucket = []  # [(step, (imgs_j, txt_j, states_j, acts_j, aux_slice_j))]
    for batch in loader:
        ep = batch["aux"]["episode_id"]
        imgs = batch["image"]
        texts = batch["text"]
        states = batch["state"]
        acts = batch["action"]
        aux = batch["aux"]

        B = imgs.size(0) if hasattr(imgs, "size") else 1
        # 리스트/텐서 → 파이썬/리스트
        if isinstance(ep, torch.Tensor): ep = ep.detach().cpu().tolist()
        if not isinstance(ep, (list,tuple)): ep = [ep]*B
        txt_list = texts if isinstance(texts, (list,tuple)) else [str(texts)]*B

        for i in range(B):
            if ep[i] != target_episode_id:
                continue
            step_i = _get_step(i, aux)
            # 개별 샘플 슬라이스 준비 (CPU 상에서 보관)
            imgs_i = imgs[i].cpu()
            states_i = states[i].cpu() if isinstance(states, torch.Tensor) else states
            acts_i = torch.as_tensor(acts[i]).cpu()
            # 텍스트는 str로
            txt_i = str(txt_list[i]) if i < len(txt_list) else str(txt_list[0])
            # 필요한 aux 일부만
            aux_i = {k: (v[i] if isinstance(v, (list,tuple)) else v) for k,v in aux.items() if k in ("file_path","episode_id")}
            bucket.append((step_i, (imgs_i, txt_i, states_i, acts_i, aux_i)))
            if len(bucket) >= max_frames: break
        if len(bucket) >= max_frames: break

    if not bucket:
        print(f"[viz-episode] episode_id={target_episode_id} 를 찾지 못했습니다.")
        vlog.close()
        return

    # 2) step 기준 정렬
    # step 정보 없는 항목(None)은 뒤로 보냄
    bucket.sort(key=lambda x: (999999999 if x[0] is None else x[0]))

    # 3) 순회하며 추론/타일화
    for step_i, pack in bucket:
        imgs_i, txt_i, states_i, acts_i, aux_i = pack
        imgs_i = imgs_i.unsqueeze(0).to(device, non_blocking=True)   # [1,3,H,W]
        states_i = states_i.unsqueeze(0).to(device, non_blocking=True) if isinstance(states_i, torch.Tensor) else states_i
        acts_i = acts_i.unsqueeze(0).to(device, non_blocking=True)

        # 모델 호출 시 your forward 시그니처에 맞추기
        try:
            pred_i = model_like(imgs_i, [txt_i], states_i)
        except TypeError:
            pred_i = model_like(imgs_i, [txt_i])  # states 미사용 모델일 때

        tile = make_preview_tile(
            images=imgs_i, texts=[txt_i], states=states_i,
            gt_actions=acts_i, pred_actions=pred_i,
            max_n=1, downscale=1.0, which=0
        )
        vlog.write_pil(tile, async_save=False)
    vlog.close()
    print(f"[viz-episode] episode_id={target_episode_id} → {out_mp4} 저장 완료")

Overwriting /content/drive/MyDrive/vla_project/utils/vis.py


In [ ]:
%%writefile /content/drive/MyDrive/vla_project/data/data.py
# /content/drive/MyDrive/vla_project/data/data.py
import os, io, json
from typing import Dict, Any, List, Iterator, Optional
import numpy as np
from PIL import Image

import torch
from torch.utils.data import IterableDataset, DataLoader

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import tensorflow_datasets as tfds


# -----------------------------
# 유틸 함수
# -----------------------------
def _tf_to_numpy(x):
    """tf.Tensor or EagerTensor -> np.ndarray or Python scalar"""
    if hasattr(x, "numpy"):
        return x.numpy()
    return x  # 이미 numpy나 파이썬 타입인 경우

def _decode_str(x):
    """bytes/tf.string -> str"""
    if hasattr(x, "numpy"):
        v = x.numpy()
        if isinstance(v, (bytes, bytearray)):
            return v.decode("utf-8", errors="ignore")
        return str(v)
    if isinstance(x, (bytes, bytearray)):
        return x.decode("utf-8", errors="ignore")
    return str(x)

def _to_image_tensor(img: np.ndarray, size: int = 224) -> torch.Tensor:
    """HxWx3 uint8 -> (3, H, W) float32[0~1]"""
    if img.ndim != 3 or img.shape[2] != 3:
        raise ValueError(f"expected HxWx3, got {img.shape}")
    pil = Image.fromarray(img).resize((size, size))
    arr = np.asarray(pil, dtype=np.float32) / 255.0
    arr = np.transpose(arr, (2, 0, 1))  # to CHW
    return torch.from_numpy(arr)

def _image_from_step(st: Dict[str, Any]) -> np.ndarray:
    """우선순위: observation.image_0 (필요시 다른 카메라도 확장 가능)"""
    obs = st.get("observation", {})
    if "image_0" in obs:
        return _tf_to_numpy(obs["image_0"])
    # fallback (데이터셋 변형용)
    if "image" in st:
        return _tf_to_numpy(st["image"])
    raise KeyError("No image key found in step (expected observation.image_0)")

def _maybe_to_torch_1d(x) -> Optional[torch.Tensor]:
    if x is None:
        return None
    x = _tf_to_numpy(x)
    x = np.asarray(x)
    if x.ndim == 0:  # scalar -> (1,)
        x = x[None]
    return torch.from_numpy(x.astype(np.float32, copy=False))


# -----------------------------
# Reasoning Lookup (선택)
# -----------------------------
class ReasoningLookup:
    """
    reasoning_json 파일 구조 예시:
    {
      "/abs/path/to/out.npy": {
        "43": {
          "features": {"move_primitive": ["stop", "pick", ...]},
          "alt_instruction": {...}
        },
        ...
      },
      ...
    }
    """
    def __init__(self, json_path: str, primitive_to_id: bool = False):
        self.tbl = {}
        self.primitive2id = None

        if not json_path or not os.path.isfile(json_path):
            print(f"[WARN] reasoning JSON not found: {json_path}")
            return

        with open(json_path, "r", encoding="utf-8") as f:
            raw = json.load(f)

        # (옵션) 프리미티브 사전 구성
        if primitive_to_id:
            uniq = set()
            for _, epdict in raw.items():
                for _, content in epdict.items():
                    for mp in content.get("features", {}).get("move_primitive", []):
                        uniq.add(mp)
            self.primitive2id = {p: i for i, p in enumerate(sorted(uniq))}
            print(f"[INFO] primitive2id built: {len(self.primitive2id)} labels")

        cnt = 0
        for file_name, epdict in raw.items():
            for ep_id, content in epdict.items():
                moves = content.get("features", {}).get("move_primitive", [])
                for i, mp in enumerate(moves):
                    key = f"{file_name}_{ep_id}_{i}"
                    if self.primitive2id is not None:
                        mp_val = self.primitive2id.get(mp, -1)
                    else:
                        mp_val = mp
                    self.tbl[key] = {
                        "move_primitive": mp_val,
                        "alt_instruction": content.get("alt_instruction", None),
                    }
                    cnt += 1
        print(f"[INFO] reasoning entries: {cnt}")

    def get(self, file_path: str, ep_id: int, idx: int):
        key = f"{file_path}_{ep_id}_{idx}"
        return self.tbl.get(key, {})


# -----------------------------
# Dataset
# -----------------------------
class VLADataset(IterableDataset):
    """
    TFDS 에피소드 구조(사용자 제공 spec 기준):

    Top-level keys: ['episode_metadata', 'steps']
    episode_metadata keys: ['episode_id', 'file_path', 'has_image_0', 'has_image_1', 'has_image_2', 'has_image_3', 'has_language']

    steps.element_spec:
      - action: (7,) float32
      - discount: () float32
      - is_first/is_last/is_terminal: () bool
      - language_embedding: (512,) float32
      - language_instruction: () string
      - observation:
          image_0..image_3: (256,256,3) uint8
          state: (7,) float32
      - reward: () float32
    """
    def __init__(
        self,
        builder_dir: str,
        split: str = "train[:80%]",
        reasoning_json: Optional[str] = None,   # 선택
        map_primitive_to_id: bool = False,      # 선택
        img_size: int = 224,
        verbose: bool = True,
    ):
        super().__init__()
        self.builder_dir = builder_dir
        self.split = split
        self.img_size = img_size
        self.verbose = verbose

        # GPU mem growth (TF)
        gpus = tf.config.list_physical_devices('GPU')
        for g in gpus:
            try:
                tf.config.experimental.set_memory_growth(g, True)
            except Exception:
                pass

        self.builder = tfds.builder_from_directory(builder_dir=self.builder_dir)

        # reasoning lookup (선택)
        self.reasoning = None
        if reasoning_json:
            self.reasoning = ReasoningLookup(reasoning_json, primitive_to_id=map_primitive_to_id)

    def __iter__(self) -> Iterator[Dict[str, Any]]:
        ds = self.builder.as_dataset(split=self.split)

        for ep in ds:
            meta = ep["episode_metadata"]
            file_path = _decode_str(meta.get("file_path", ""))
            ep_id = int(_tf_to_numpy(meta.get("episode_id", 0)))

            steps = ep["steps"]
            # TF 2 eager에서는 파이썬 for-iter 지원. 환경에 따라 as_numpy_iterator() 사용 가능.
            for i, st in enumerate(steps):
                # 이미지
                img_np = _image_from_step(st)
                img = _to_image_tensor(img_np, self.img_size)

                # 언어 지시문
                instr = _decode_str(st.get("language_instruction", b""))

                # 언어 임베딩 (512,)
                lang_emb = st.get("language_embedding", None)
                lang_emb_t = _maybe_to_torch_1d(lang_emb)

                # 상태 (7,)
                state = st.get("observation", {}).get("state", None)
                state_t = _maybe_to_torch_1d(state)

                # 액션 (7,)
                action = st.get("action", None)
                action_t = _maybe_to_torch_1d(action)

                # 보상/디스카운트/플래그
                reward = st.get("reward", None)
                discount = st.get("discount", None)
                is_first = st.get("is_first", None)
                is_last = st.get("is_last", None)
                is_terminal = st.get("is_terminal", None)

                reward_v = float(_tf_to_numpy(reward)) if reward is not None else 0.0
                discount_v = float(_tf_to_numpy(discount)) if discount is not None else 1.0
                is_first_v = bool(_tf_to_numpy(is_first)) if is_first is not None else False
                is_last_v = bool(_tf_to_numpy(is_last)) if is_last is not None else False
                is_terminal_v = bool(_tf_to_numpy(is_terminal)) if is_terminal is not None else False

                # reasoning 매칭 (move_primitive 등)
                move_primitive = None
                if self.reasoning:
                    r = self.reasoning.get(file_path, ep_id, i)
                    move_primitive = r.get("move_primitive", None)
                    # instruction이 비어있고 대체 문구가 있으면 Caption 사용
                    if not instr and isinstance(r.get("alt_instruction"), dict):
                        instr = r["alt_instruction"].get("Caption", "") or ""

                yield dict(
                    image=img,                               # (3,H,W) float32
                    instruction=instr or "",                 # str
                    language_embedding=lang_emb_t,           # (512,) float32 or None
                    action=action_t,                         # (7,) float32 or None
                    state=state_t,                           # (7,) float32 or None
                    reward=reward_v,                         # float
                    discount=discount_v,                     # float
                    is_first=is_first_v,                     # bool
                    is_last=is_last_v,                       # bool
                    is_terminal=is_terminal_v,               # bool
                    move_primitive=move_primitive,           # str or int or None
                    file_path=file_path,                     # str
                    episode_id=ep_id,                        # int
                    step_idx=i,                              # int
                )


# -----------------------------
# Collate
# -----------------------------
def _stack_or_none(items: List[Optional[torch.Tensor]]) -> Optional[torch.Tensor]:
    """리스트에서 None 제외하고 stack. 전부 None이면 None."""
    buf = [x for x in items if x is not None]
    if not buf:
        return None
    return torch.stack(buf, dim=0)

def collate_vla(batch: List[Dict[str, Any]]):
    # 필수
    imgs = torch.stack([b["image"] for b in batch])  # (B,3,H,W)
    texts = [b["instruction"] for b in batch]
    moves = [b["move_primitive"] for b in batch]

    # 선택 텐서들
    lang_embs = _stack_or_none([b.get("language_embedding") for b in batch])  # (B,512) or None
    actions   = _stack_or_none([b.get("action") for b in batch])              # (B,7) or None
    states    = _stack_or_none([b.get("state") for b in batch])               # (B,7) or None

    rewards   = torch.tensor([b.get("reward", 0.0) for b in batch], dtype=torch.float32)
    discounts = torch.tensor([b.get("discount", 1.0) for b in batch], dtype=torch.float32)
    is_first  = torch.tensor([1.0 if b.get("is_first", False) else 0.0 for b in batch], dtype=torch.float32)
    is_last   = torch.tensor([1.0 if b.get("is_last", False) else 0.0 for b in batch], dtype=torch.float32)
    is_term   = torch.tensor([1.0 if b.get("is_terminal", False) else 0.0 for b in batch], dtype=torch.float32)

    aux = {
        "file_path": [b["file_path"] for b in batch],
        "episode_id": [b["episode_id"] for b in batch],
        "step_idx": [b["step_idx"] for b in batch],
    }

    return {
        "image": imgs,
        "text": texts,
        "move_primitive": moves,          # str list or int list (ReasoningLookup 설정에 따라)
        "language_embedding": lang_embs,  # or None
        "action": actions,                # or None
        "state": states,                  # or None
        "reward": rewards,
        "discount": discounts,
        "is_first": is_first,
        "is_last": is_last,
        "is_terminal": is_term,
        "aux": aux,
    }


# -----------------------------
# make_dataloaders
# -----------------------------
def make_dataloaders(cfg):
    """
    cfg 필수/옵션:
      - cfg.builder_dir (str)
      - cfg.img_size (int)
      - cfg.batch_size (int)
      - cfg.train_split (str, optional, default "train[:80%]")
      - cfg.val_split   (str, optional, default "train[80%:]")
      - cfg.reasoning_json (str or None)
      - cfg.map_primitive_to_id (bool, default False)
      - cfg.num_workers (int, default 0)
      - cfg.shuffle_buffer (int, optional; IterableDataset에서는 보통 외부 셔플러 필요)
    """
    train_ds = VLADataset(
        builder_dir=cfg.builder_dir,
        split=getattr(cfg, "train_split", "train[:80%]"),
        reasoning_json=getattr(cfg, "reasoning_json", None),
        map_primitive_to_id=getattr(cfg, "map_primitive_to_id", False),
        img_size=cfg.img_size,
    )
    val_ds = VLADataset(
        builder_dir=cfg.builder_dir,
        split=getattr(cfg, "val_split", "train[80%:]"),
        reasoning_json=getattr(cfg, "reasoning_json", None),
        map_primitive_to_id=getattr(cfg, "map_primitive_to_id", False),
        img_size=cfg.img_size,
    )

    nw = int(getattr(cfg, "num_workers", 0))
    train_loader = DataLoader(
        train_ds, batch_size=cfg.batch_size, num_workers=nw, collate_fn=collate_vla
    )
    val_loader = DataLoader(
        val_ds, batch_size=cfg.batch_size, num_workers=nw, collate_fn=collate_vla
    )
    return train_loader, val_loader

Overwriting /content/drive/MyDrive/vla_project/data/data.py


In [ ]:
# file :  models/vision.py

%%writefile /content/drive/MyDrive/vla_project/models/vision.py
import torch.nn as nn
import timm

class DinoV2Encoder(nn.Module):
    def __init__(self, model_name: str, embed_dim: int):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=0)
        out_dim = getattr(self.backbone, "num_features", embed_dim)
        self.proj = nn.Identity() if out_dim == embed_dim else nn.Linear(out_dim, embed_dim)

    def forward(self, x):
        feats = self.backbone(x)   # [B, C]
        return self.proj(feats)

Overwriting /content/drive/MyDrive/vla_project/models/vision.py


In [ ]:
# 디렉토리 생성
%%bash
mkdir -p /content/drive/MyDrive/vla_project/models/llm_func

In [ ]:
# models/llm_func/__init__.py
%%writefile /content/drive/MyDrive/vla_project/models/llm_func/__init__.py
# llm_func package

Overwriting /content/drive/MyDrive/vla_project/models/llm_func/__init__.py


In [ ]:
# models/llm_func/lora_ft.py — LLaMA 백본만 LoRA-FT (클래스 버전)
%%writefile /content/drive/MyDrive/vla_project/models/llm_func/lora_ft.py
from typing import Iterable, Tuple, Optional, List
import torch.nn as nn

try:
    from peft import LoraConfig, get_peft_model, PeftModel
except Exception as e:
    raise RuntimeError("peft가 필요합니다. `pip install peft`") from e


DEFAULT_LLAMA_TARGET_MODULES: List[str] = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
]


class LoraFTManager:
    """
    - 전체 모델을 동결하고, model.text.backbone(HF LLaMA) 에만 LoRA 부착
    - 필요한 경우 GC(gradient checkpointing) 활성화
    """
    def __init__(
        self,
        model: nn.Module,
        *,
        target_modules: Optional[Iterable[str]] = None,
        bias: str = "none",
        task_type: str = "CAUSAL_LM"
    ):
        self.model = model
        self.target_modules = list(target_modules) if target_modules is not None else list(DEFAULT_LLAMA_TARGET_MODULES)
        self.bias = bias
        self.task_type = task_type

    # --------- public API ---------
    def freeze_all(self) -> None:
        self._set_requires_grad(self.model, False)

    def attach(
        self,
        *,
        r: int = 8,
        alpha: int = 32,
        dropout: float = 0.1,
        gradient_checkpointing: bool = True,
        print_report: bool = True,
    ) -> nn.Module:
        """LoRA를 LLaMA 백본에만 부착하고 나머지는 동결."""
        self.freeze_all()
        llama = self._get_llama_backbone()

        cfg = LoraConfig(
            r=r,
            lora_alpha=alpha,
            lora_dropout=dropout,
            target_modules=self.target_modules,
            bias=self.bias,
            task_type=self.task_type,
        )
        lora_llama = get_peft_model(llama, cfg)

        if gradient_checkpointing and hasattr(lora_llama, "gradient_checkpointing_enable"):
            try:
                lora_llama.gradient_checkpointing_enable()
            except Exception:
                pass

        # VLA에 장착
        self._set_text_backbone(lora_llama)

        if print_report:
            try:
                lora_llama.print_trainable_parameters()
            except Exception:
                tr, fr = self.count_params()
                print(f"[LoRA] trainable={tr:,} frozen={fr:,}")

        return self.model

    def is_attached(self) -> bool:
        try:
            return isinstance(self._get_llama_backbone(), PeftModel)
        except Exception:
            return False

    def save_adapters(self, save_dir: str) -> None:
        """LoRA 어댑터만 저장."""
        llm = self._get_llama_backbone()
        if not isinstance(llm, PeftModel):
            raise RuntimeError("LoRA가 부착되어 있지 않습니다.")
        llm.save_pretrained(save_dir)
        print(f"[LoRA] adapters saved to: {save_dir}")

    def count_params(self) -> Tuple[int, int]:
        trainable, frozen = 0, 0
        for p in self.model.parameters():
            (trainable if p.requires_grad else frozen).__iadd__(p.numel())
        return trainable, frozen

    # --------- internals ---------
    @staticmethod
    def _set_requires_grad(module: nn.Module, flag: bool) -> None:
        for p in module.parameters():
            p.requires_grad = flag

    def _get_llama_backbone(self) -> nn.Module:
        """
        VLA 구현에서 텍스트 백본을 model.text.backbone 으로 가정.
        다르면 이 메서드를 수정하세요.
        """
        text = getattr(self.model, "text", None)
        if text is None:
            raise RuntimeError("model.text 를 찾을 수 없습니다. VLA 구현을 확인하세요.")
        backbone = getattr(text, "backbone", None)
        if backbone is None:
            raise RuntimeError("model.text.backbone 을 찾을 수 없습니다. VLA 구현을 확인하세요.")
        return backbone

    def _set_text_backbone(self, new_backbone: nn.Module) -> None:
        text = getattr(self.model, "text", None)
        if text is None:
            raise RuntimeError("model.text 를 찾을 수 없습니다.")
        setattr(text, "backbone", new_backbone)

Overwriting /content/drive/MyDrive/vla_project/models/llm_func/lora_ft.py


In [ ]:
# file: models/text.py

%%writefile /content/drive/MyDrive/vla_project/models/text.py


import torch, torch.nn as nn
from transformers import AutoTokenizer, AutoModel

class LlamaTextEncoder(nn.Module):
    def __init__(self, model_name: str, embed_dim: int,
                 fp16: bool = True, pad_token_str: str = "<|pad|>",
                 max_len: int = 256):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({"pad_token": pad_token_str})
        self.max_len = int(max_len)
        self.tokenizer.model_max_length = self.max_len

        dtype = torch.float16 if fp16 else None
        self.backbone = AutoModel.from_pretrained(model_name, dtype=dtype)

        if len(self.tokenizer) != self.backbone.get_input_embeddings().num_embeddings:
            self.backbone.resize_token_embeddings(len(self.tokenizer))

        self.hidden_size = self.backbone.config.hidden_size
        self.embed_dim   = embed_dim
        self.proj = nn.Linear(self.hidden_size, self.embed_dim, bias=False) \
                    if self.hidden_size != self.embed_dim else nn.Identity()

    def forward(self, texts):
        # texts: list[str] 보장 (VLA에서 강제)
        if isinstance(texts, (list, tuple)):
            txt_list = [t if isinstance(t, str) else str(t) for t in texts]
        else:
            txt_list = [str(texts)]

        enc = self.tokenizer(
            txt_list, padding=True, truncation=True,
            max_length=self.max_len, return_tensors="pt"
        )
        device = next(self.backbone.parameters()).device
        enc = {k: v.to(device, non_blocking=True) for k, v in enc.items()}

        if not hasattr(self, "_dbg_tok"):
            print(f"[DBG/TEXT] input_ids={tuple(enc['input_ids'].shape)}", flush=True)
            self._dbg_tok = True

        out = self.backbone(**enc)                 # [B, T, H]
        t_feat = out.last_hidden_state.mean(dim=1) # [B, H]
        t_feat = self.proj(t_feat)                 # [B, embed_dim]
        return t_feat

Overwriting /content/drive/MyDrive/vla_project/models/text.py


In [ ]:
#file models/policy.py

%%writefile /content/drive/MyDrive/vla_project/models/policy.py
import torch.nn as nn
import torch

class PolicyHead(nn.Module):
    def __init__(self, vision_dim: int, text_dim: int, state_dim: int, hidden_dim: int, action_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(vision_dim + text_dim + state_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, action_dim),
        )
    def forward(self, v, t , s):

        x = torch.cat([v, t , s], dim=-1)
        return self.net(x)

Overwriting /content/drive/MyDrive/vla_project/models/policy.py


In [ ]:
#file models/vla.py

%%writefile /content/drive/MyDrive/vla_project/models/vla.py


import torch
import torch.nn as nn
from models.vision import DinoV2Encoder
from models.text import LlamaTextEncoder
from models.policy import PolicyHead

class VLA(nn.Module):
    def __init__(self,
                 vision_backbone: str, vision_embed_dim: int,
                 text_model_name: str, text_embed_dim: int,
                 state_dim: int,
                 hidden_dim: int, action_dim: int,
                 fp16_text: bool = True, pad_token_str: str = "<|pad|>",
                 text_max_len: int = 256):
        super().__init__()
        self.vision = DinoV2Encoder(model_name=vision_backbone, embed_dim=vision_embed_dim)
        self.text   = LlamaTextEncoder(model_name=text_model_name, embed_dim=text_embed_dim,
                                       fp16=fp16_text, pad_token_str=pad_token_str,
                                       max_len=text_max_len)
        self.policy = PolicyHead(
            vision_dim=vision_embed_dim,
            text_dim=text_embed_dim,
            state_dim = state_dim,
            hidden_dim=hidden_dim,
            action_dim=action_dim,
        )

    def forward(self, images, texts , states):

        B = images.size(0)
        v = self.vision(images)  # [B, V]
        texts = self.text(texts)  # [B, Tdim]
        return self.policy(v, texts , states)

Overwriting /content/drive/MyDrive/vla_project/models/vla.py


In [ ]:
# loss.py


%%writefile /content/drive/MyDrive/vla_project/losses.py
import torch.nn.functional as F

import torch
def action_loss(logits: torch.Tensor, target: torch.Tensor):
    # logits: [B, C], target: [B, C] (float)
    if target.dtype != logits.dtype:
        target = target.to(logits.dtype)
    assert target.shape == logits.shape, f"shape mismatch: {logits.shape} vs {target.shape}"
    return F.mse_loss(logits, target)

Overwriting /content/drive/MyDrive/vla_project/losses.py


In [ ]:
%%bash
mkdir -p /content/drive/MyDrive/vla_project/{trainers,utils,data}

#touch /content/drive/MyDrive/vla_project/trainers/__init__.py
#touch /content/drive/MyDrive/vla_project/utils/__init__.py
#touch /content/drive/MyDrive/vla_project/dataio/__init__.py

In [ ]:
# utils/progress.py — 진행상황 계산 유틸들
%%writefile /content/drive/MyDrive/vla_project/utils/progress.py
import torch, math, time

class SmoothedValue:
    def __init__(self, momentum: float = 0.98):
        self.m = None
        self.beta = momentum
    def update(self, x: float):
        if x is None: return
        self.m = x if self.m is None else (self.beta * self.m + (1 - self.beta) * x)
    @property
    def value(self):
        return float("nan") if self.m is None else float(self.m)

def get_lr(optimizer=None, ds_engine=None):
    if ds_engine is not None:
        try:
            pg = ds_engine.optimizer.param_groups
            if pg and "lr" in pg[0]:
                return pg[0]["lr"]
        except Exception:
            pass
        try:
            vals = ds_engine.get_lr()
            return vals[0] if isinstance(vals, (list, tuple)) and vals else float("nan")
        except Exception:
            return float("nan")
    if optimizer is None: return float("nan")
    try:
        return optimizer.param_groups[0]["lr"]
    except Exception:
        return float("nan")

def gpu_mem_gb():
    if not torch.cuda.is_available(): return 0.0, 0.0
    alloc = torch.cuda.memory_allocated() / (1024**3)
    reserved = torch.cuda.memory_reserved() / (1024**3)
    return alloc, reserved

Overwriting /content/drive/MyDrive/vla_project/utils/progress.py


In [ ]:
# trainers/eval.py — 평가 함수
%%writefile /content/drive/MyDrive/vla_project/utils/eval.py
import torch
from tqdm import tqdm
from losses import action_loss

@torch.no_grad()
def evaluate(model_like, loader, device, use_ds: bool):
    """
    - model_like: DeepSpeed engine 또는 일반 nn.Module
    - loader: data.py의 collate_vla가 반환한 배치(dict)용 DataLoader
    """
    # eval 모드 전환 (파라미터는 module 쪽에서 관리)
    module = model_like.module if use_ds else model_like
    was_training = module.training
    module.eval()

    total, n = 0.0, 0
    pbar = tqdm(loader, desc="[eval]", leave=False)

    for batch in pbar:
        imgs   = batch["image"].to(device, non_blocking=True)
        texts  = batch.get("texts", batch.get("text"))
        states = batch.get("state")
        acts   = batch.get("action")

        # 선택 텐서 처리
        states = states.to(device, non_blocking=True) if states is not None else None
        if acts is None:
            # 손실 계산 불가 → 스킵
            pbar.set_postfix(skip="no_action")
            continue
        acts = acts.to(device, non_blocking=True)

        # forward (states 유무에 따라)
        if states is None:
            pred = model_like(imgs, texts) if use_ds else module(imgs, texts)
        else:
            pred = model_like(imgs, texts, states) if use_ds else module(imgs, texts, states)

        loss = action_loss(pred, acts)

        bs = imgs.size(0)
        total += loss.item() * bs
        n     += bs
        pbar.set_postfix(loss=f"{loss.item():.4f}", avg=f"{(total/max(1,n)):.4f}")

    # 원래 모드 복귀
    if was_training:
        module.train()
    else:
        module.eval()

    return total / max(1, n)

Overwriting /content/drive/MyDrive/vla_project/utils/eval.py


In [ ]:
# utils/util.py — set_seed, save_checkpoint
%%writefile /content/drive/MyDrive/vla_project/utils/util.py
import os, io, random, tempfile
from typing import Any, Dict, Union
import torch
import numpy as np

def _atomic_write_bytes(data: bytes, path: str) -> None:
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    dir_ = os.path.dirname(path) or "."
    fd, tmppath = tempfile.mkstemp(prefix=".tmp_", dir=dir_)
    try:
        with os.fdopen(fd, "wb") as f:
            f.write(data)
            f.flush()
            os.fsync(f.fileno())
        os.replace(tmppath, path)  # atomic on POSIX
    finally:
        try:
            if os.path.exists(tmppath):
                os.remove(tmppath)
        except Exception:
            pass

Overwriting /content/drive/MyDrive/vla_project/utils/util.py


In [ ]:
# -----------------------------
# file: trainer.py
# -----------------------------

%%writefile /content/drive/MyDrive/vla_project/trainer.py

import os, json, time
os.environ.setdefault("PYTHONUNBUFFERED", "1")      # 즉시 flush
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")  # TF 로그 숨김

import torch
import deepspeed
from tqdm import tqdm
from torch.amp import autocast, GradScaler

from models.vla import VLA
from losses import action_loss

from util import set_seed , save_checkpoint
from utils.progress import SmoothedValue, get_lr, gpu_mem_gb
from utils.eval import evaluate
from utils.vis import viz_episode

from models.llm_func.lora_ft import LoraFTManager

# 선택: 성능/안정성
try:
    torch.backends.cudnn.benchmark = True
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

def train_loop(cfg):
    set_seed(cfg.seed)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ------------------------------
    # Data
    # ------------------------------
    from data.data import make_dataloaders
    train_loader, val_loader = make_dataloaders(cfg)

    # ------------------------------
    # Model
    # ------------------------------
    model = VLA(
        cfg.vision_backbone, cfg.vision_embed_dim,
        cfg.text_model_name, cfg.text_embed_dim,
        cfg.state_dim,
        cfg.hidden_dim, cfg.action_dim,
        fp16_text=cfg.amp, pad_token_str=cfg.pad_token_str,
        text_max_len=cfg.text_max_len
    )

    # ------------------------------
    # LoRA attach (텍스트 백본만 학습)
    # ------------------------------
    lora_mgr = LoraFTManager(model, task_type="FEATURE_EXTRACTION")
    model = lora_mgr.attach(
        r=getattr(cfg, "lora_r", 8),
        alpha=getattr(cfg, "lora_alpha", 32),
        dropout=getattr(cfg, "lora_dropout", 0.1),
        gradient_checkpointing=True,
        print_report=True,
    )

    # ------------------------------
    # Engine / Optimizer
    # ------------------------------
    use_ds   = bool(getattr(cfg, "use_deepspeed", False))
    grad_clip = float(getattr(cfg, "grad_clip", 0.0) or 0.0)

    if use_ds:
        with open(cfg.deepspeed_config, 'r') as f:
            ds_cfg = json.load(f)
        ds_cfg.setdefault('train_batch_size', cfg.batch_size)
        ds_cfg.setdefault('gradient_accumulation_steps', cfg.grad_accumulation_steps)

        model_engine, optimizer, _, _ = deepspeed.initialize(
            model=model,
            model_parameters=[p for p in model.parameters() if p.requires_grad],
            config=ds_cfg,
        )
        device = model_engine.device
        scaler = None
        forward_obj = model_engine
    else:
        model = model.to(device)
        trainables = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.AdamW(trainables, lr=cfg.lr, weight_decay=cfg.weight_decay)
        # torch.amp 권고 API로 변경(FutureWarning 해결)
        scaler = GradScaler(enabled=cfg.amp, device="cuda")
        forward_obj = model

    # ------------------------------
    # Loop
    # ------------------------------
    global_step = 0
    os.makedirs(cfg.ckpt_dir, exist_ok=True)

    loss_avg = SmoothedValue()
    t0 = time.time()
    samples_processed = 0
    last_eval_loss = None

    steps_per_epoch = int(getattr(cfg, "steps_per_epoch", 2000))

    for epoch in range(cfg.epochs):

        it_start = time.time()

        # 배치 타이밍 분해 측정 변수
        load_t = fwd_t = bwd_t = 0.0

        # enumerate로 인덱스, 강제 pbar.update(1)
        for batch_idx, batch in enumerate(train_loader, 1):
            t_load0 = time.time()
            global_step += 1

            texts = batch['text']
            imgs = batch['image']
            states = batch['state'].to(device)
            rewards = batch['reward'].to(device)
            discounts = batch['discount'].to(device)
            is_first = batch['is_first'].to(device)
            is_last = batch['is_last'].to(device)
            is_term = batch['is_terminal'].to(device)
            aux = batch['aux']

            bsz = imgs.size(0) if hasattr(imgs, "size") else cfg.batch_size

            imgs   = imgs.to(device, non_blocking=True)

            acts = torch.tensor(batch['action'])
            acts = acts.to(device, non_blocking=True)

            t_load1 = time.time()

            # fwd/bwd
            if use_ds:
                t_fwd0 = time.time()
                pred = forward_obj(imgs, texts)
                loss = action_loss(pred, acts)
                t_fwd1 = time.time()
                forward_obj.backward(loss)
                forward_obj.step()
                t_bwd1 = time.time()
            else:
                optimizer.zero_grad(set_to_none=True)
                t_fwd0 = time.time()
                # torch.amp 권고 API
                with torch.amp.autocast("cuda", enabled=cfg.amp):
                    pred = forward_obj(imgs, texts , states)
                    loss = action_loss(pred, acts)

                t_fwd1 = time.time()
                scaler.scale(loss).backward()

                if grad_clip > 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(forward_obj.parameters(), grad_clip)
                scaler.step(optimizer)
                scaler.update()
                t_bwd1 = time.time()

            # 타이밍 누적
            load_t += (t_load1 - t_load0)
            fwd_t  += (t_fwd1 - t_fwd0)
            bwd_t  += (t_bwd1 - t_fwd1)

            # 진행도/속도
            loss_avg.update(loss.item())
            samples_processed += bsz
            elapsed = max(1e-6, time.time() - t0)
            steps_per_s   = global_step / elapsed
            samples_per_s = samples_processed / elapsed
            lr = get_lr(optimizer if not use_ds else None, forward_obj if use_ds else None)
            alloc, reserved = gpu_mem_gb()

            # 주기적 로그 (flush 강제)
            if global_step % cfg.log_interval == 0:
                lr_str = f"{lr:.2e}" if isinstance(lr, (float, int)) else str(lr)
                print(
                      f"[log] ep {epoch} st {global_step} "
                      f"loss={loss.item():.4f} avg={loss_avg.value:.4f} "
                      f"lr={lr_str} "
                      f"mem={alloc:.2f}/{reserved:.2f}GB "
                      f"load={load_t/(batch_idx):.3f}s fwd={fwd_t/(batch_idx):.3f}s bwd={bwd_t/(batch_idx):.3f}s "
                      f"sp_s={steps_per_s:.2f} samp_s={samples_per_s:.1f}",
                      flush=True,
                    )

            #if global_step % cfg.eval_interval == 0:
            #    model_like = forward_obj
            #    last_eval_loss = evaluate(model_like, val_loader, device, use_ds)
            #    print(f"[eval] global_step {global_step} | val_loss={last_eval_loss:.4f}", flush=True)

            if global_step % cfg.save_interval == 0:
                target = forward_obj.module if use_ds else forward_obj
                path = os.path.join(cfg.ckpt_dir, f"vla_step{global_step}.pt")
                save_checkpoint(path, target.state_dict())
                print(f"[save] step {global_step} -> {path}", flush=True)
                # LoRA 어댑터만 별도 저장(옵션)
                try:
                    from peft import PeftModel
                    if isinstance(target.text.backbone, PeftModel):
                        target.text.backbone.save_pretrained(os.path.join(cfg.ckpt_dir, f"lora_step{global_step}"))
                except Exception:
                    pass

            if batch_idx >= steps_per_epoch:
              break

        # 시각화


        epoch_time = time.time() - it_start
        if last_eval_loss is not None:
            print(f"[epoch] {epoch} done in {epoch_time:.1f}s | avg_loss={loss_avg.value:.4f} | last_val={last_eval_loss:.4f}", flush=True)
        else:
            print(f"[epoch] {epoch} done in {epoch_time:.1f}s | avg_loss={loss_avg.value:.4f}", flush=True)

    target = forward_obj.module if use_ds else forward_obj
    path = os.path.join(cfg.ckpt_dir, "vla_final.pt")
    save_checkpoint(path, target.state_dict())
    print(f"[done] Training finished. Saved: {path}", flush=True)

Overwriting /content/drive/MyDrive/vla_project/trainer.py


In [ ]:
# train.py

%%writefile /content/drive/MyDrive/vla_project/train.py

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# 메인은 얇게: 외부 모듈을 임포트해서 실행만 담당
from trainer import train_loop

# 필요시: config가 있다면 불러와 바로 실행 가능
if __name__ == "__main__":
    try:
        from config import TrainConfig
        cfg = TrainConfig()
        train_loop(cfg)
    except ImportError:
        print("config.TrainConfig 를 찾을 수 없습니다. 외부에서 train_loop(cfg)로 호출하세요.")

Overwriting /content/drive/MyDrive/vla_project/train.py


In [ ]:
# -----------------------------
# file: README_snippet.txt (참고)
# -----------------------------
# 1) 설치
# pip install timm transformers accelerate safetensors deepspeed
#
# 2) ds_config.json 예시
# {
# "train_batch_size": 4,
# "gradient_accumulation_steps": 2,
# "fp16": {"enabled": true}, // 또는 "bf16": {"enabled": true}
# "zero_optimization": {"stage": 2},
# "optimizer": {"type": "AdamW", "params": {"lr": 3e-4, "betas": [0.9,0.999], "eps": 1e-8, "weight_decay": 1e-4}},
# "gradient_clipping": 1.0,
# "steps_per_print": 200
# }
#
# 3) 단일 GPU 실행
# python train.py --train_csv data/train.csv --val_csv data/val.csv --image_root . \
# --vision_backbone vit_base_patch14_dinov2 --text_model_name meta-llama/Llama-3-8B-Instruct \
# --img_size 224 --batch_size 4 --amp
#
# 4) DeepSpeed 실행 (예: 1~N GPU)
# deepspeed train.py --deepspeed --deepspeed_config ds_config.json \
# --train_csv data/train.csv --val_csv data/val.csv --image_root . \
# --vision_backbone vit_base_patch14_dinov2 --text_model_name meta-llama/Llama-3-8B-Instruct \
# --img_size 224 --batch_size 4 --grad_accumulation_steps 2
#
# 5) 메모
# - 이미지 I/O: DataLoader는 num_workers=0로 두고, collate에서 ThreadPoolExecutor로 배치 단위 병렬 디코드.
# - CUDA 전송: pin_memory=True로 H2D 전송 가속, non_blocking=True 사용.
# - DeepSpeed: model_engine(imgs, texts, states) 그대로 호출 → backward/step은 engine 메서드 사용.
# - 체크포인트: DeepSpeed 모드에선 engine.module.state_dict() 저장.

In [ ]:
%%writefile /content/drive/MyDrive/vla_project/ds_config.json

{
  "train_batch_size": 4,
  "gradient_accumulation_steps": 2,
  "fp16": { "enabled": true },
  "zero_optimization": { "stage": 2 },
  "optimizer": {
    "type": "AdamW",
    "params": { "lr": 3e-4, "betas": [0.9, 0.999], "eps": 1e-8, "weight_decay": 1e-4 }
  },
  "gradient_clipping": 1.0,
  "steps_per_print": 200
}

Overwriting /content/drive/MyDrive/vla_project/ds_config.json


In [ ]:
%cd /content/drive/MyDrive/vla_project

!python train.py \
  --train_csv /content/drive/MyDrive/vla_datasets/export_vla/train.jsonl \
  --val_csv   /content/drive/MyDrive/vla_datasets/export_vla/val.jsonl \
  --image_root /content/drive/MyDrive/vla_datasets \
  --vision_backbone vit_base_patch14_dinov2 \
  --text_model_name meta-llama/Meta-Llama-3-8B-Instruct \
  --img_size 518 --batch_size 32 --amp




"""
# deepseed

%cd /content/vla_project
!deepspeed --num_gpus=1 train.py --deepspeed --deepspeed_config ds_config.json \
  --train_csv data/train.csv --val_csv data/val.csv --image_root . \
  --vision_backbone vit_base_patch14_dinov2 \
  --text_model_name meta-llama/Meta-Llama-3-8B-Instruct \
  --img_size 224 --batch_size 2 --grad_accumulation_steps 2 --amp

"""

/content
E0000 00:00:1762303963.741051    1433 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762303963.747641    1433 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762303963.764479    1433 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762303963.764511    1433 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762303963.764515    1433 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762303963.764517    1433 computation_placer.cc:177] computation placer already registered. Please ch

'\n# deepseed\n\n%cd /content/vla_project\n!deepspeed --num_gpus=1 train.py --deepspeed --deepspeed_config ds_config.json   --train_csv data/train.csv --val_csv data/val.csv --image_root .   --vision_backbone vit_base_patch14_dinov2   --text_model_name meta-llama/Meta-Llama-3-8B-Instruct   --img_size 224 --batch_size 2 --grad_accumulation_steps 2 --amp\n\n'

시각화

In [ ]:
from IPython.display import Video, display
display(Video("/content/drive/MyDrive/viz/run.mp4", embed=True))